# Prepare IWSLT12 data
Convert the data from XML to raw text


Used some code from https://github.com/attilanagy234/neural-punctuator

In [1]:
import os
import xml.etree.ElementTree as ET
import re

In [2]:
dataPath = "Data/"
rawPath = "RAW/"

In [3]:
def prepareData(sourcePath, targetPath):
    # parse xml
    xmlp = ET.XMLParser(encoding="utf-8")
    tree = ET.parse(sourcePath, parser=xmlp)
    root = tree.getroot()

    # add segments
    docs = []
    for doc_id in range(len(root[0])):
        doc_segs = []
        doc = root[0][doc_id]
        for seg in doc.iter('seg'):
            doc_segs.append(seg.text)
        docs.append(doc_segs)

    # strip text
    texts = [re.sub(r'\s+', ' ', ''.join(d)).strip() for d in docs]

    # write to file
    with open(targetPath, 'w', encoding='utf-8') as f:
        for text in texts:
            f.write(text + '\n')

In [4]:
prepareData(rawPath + "IWSLT12.TALK.dev2010.en-fr.en.xml", dataPath + "dev_texts.txt")
prepareData(rawPath + "IWSLT12.TED.MT.tst2012.en-fr.en.xml", dataPath + "test_texts_2012.txt")

In [5]:
# train data is in a slightly other format
""" Wrap original file to make it processable with defualt python parser
<?xml version="1.0" encoding="UTF-8"?>
<mteval>
...
</mteval>
"""

trainPath = rawPath + "train.tags.en-fr.en"
targetPath = dataPath + "train_texts.txt"

xmlp = ET.XMLParser(encoding="utf-8")
tree = ET.parse(trainPath, parser=xmlp)
root = tree.getroot()

# add segments
docs = []
for doc in root.iter('transcript'):
    docs.append(doc.text)

# strip text
texts = [re.sub(r'\s+', ' ', d.replace('\n', ' ')).strip() for d in docs]

# write to file
with open(targetPath, 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text + '\n')